# TiTiler-CMR: HLS Data Demo

#### Requirements
- folium
- httpx

`!pip install folium httpx earthaccess`

In [ ]:
import json
import httpx
import earthaccess

from folium import Map, TileLayer

In [ ]:
titiler_endpoint = "http://localhost:8000"

In [ ]:
datasets = earthaccess.search_datasets(doi="10.5067/HLS/HLSL30.002")
ds = datasets[0]

In [ ]:
concept_id = ds["meta"]["concept-id"]
print("Concept-Id: ", concept_id)

In [ ]:
import earthaccess
import morecantile

tms = morecantile.tms.get("WebMercatorQuad")

bounds = tms.bounds(62, 44, 7)
xmin, ymin, xmax, ymax = (round(n, 8) for n in bounds)

results = earthaccess.search_data(
    bounding_box=(xmin, ymin, xmax, ymax),
    count=1,
    concept_id=concept_id,
    temporal=("2024-02-11", "2024-02-13"),
)
print("Granules:")
print(results)
print()
print("Example of COGs URL: ")
for link in results[0].data_links(access="direct"):
    print(link)


In [ ]:
print(results[0])

In [ ]:
from titiler.cmr.backend import CMRBackend
from titiler.cmr.reader import MultiFilesBandsReader

with CMRBackend(reader=MultiFilesBandsReader) as backend:
    assets = backend.assets_for_tile(
        x=62,
        y=44,
        z=7,
        bands_regex="B[0-9][0-9]",
        concept_id=concept_id,
        temporal=("2024-02-11", "2024-02-13")
    )

print(assets[0])

In [ ]:
with CMRBackend() as backend:
    assets = backend.assets_for_tile(
        x=62,
        y=44,
        z=7,
        bands_regex="B[0-9][0-9]",
        concept_id="C2021957657-LPCLOUD",
        temporal=("2024-02-11", "2024-02-13")
    )


In [ ]:
from IPython.display import IFrame
IFrame(f"{titiler_endpoint}/api.html", 900,500)

In [ ]:
r = httpx.get(
    f"{titiler_endpoint}/WebMercatorQuad/tilejson.json",
    params = (
        ("concept_id", concept_id),
        # Datetime in form of `start_date/end_date`
        ("datetime", "2024-10-01T00:00:00Z/2024-10-10T23:59:59Z"),
        # We know that the HLS collection dataset is stored as File per Band
        # so we need to pass a `band_regex` option to assign `bands` to each URL
        ("bands_regex", "B[0-9][0-9]"),
        # titiler-cmr can work with both Zarr and COG dataset
        # but we need to tell the endpoints in advance which backend
        # to use
        ("backend", "rasterio"),
        # True Color Image B04,B03,B02
        ("bands", "B04"),
        ("bands", "B03"),
        ("bands", "B02"),
        # The data is in type of Uint16 so we need to apply some
        # rescaling/color_formula in order to create PNGs
        # ("rescale", "0,1000"),
        ("color_formula", "Gamma RGB 3.5 Saturation 1.7 Sigmoidal RGB 15 0.35"),
        # We need to set min/max zoom because we don't want to use lowerzoom level (e.g 0)
        # which will results in useless large scale query
        ("minzoom", 8),
        ("maxzoom", 13),
    )
).json()

print(r)

In [ ]:
bounds = r["bounds"]
m = Map(
    location=(47.590266824611675, -91.03729840730689),
    zoom_start=r["maxzoom"] - 3
)

TileLayer(
    tiles=r["tiles"][0],
    opacity=1,
    attr="NASA",
).add_to(m)
m